In [1]:
import sys, os
import numpy as np
from math import acos
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [2]:
def load_data(path):
    X=[]
    Y=[]
    # 101 faces - sub folders, 6 emotions - Angry, Disgust, Fear, Happy, Sad, Surprise
    for face_dir in os.listdir(path)[1:]:
        face_path = path+face_dir+'/'
        for label in os.listdir(face_path):
            emotion_path = face_path+label+'/'
            for file in os.listdir(emotion_path):
                if file.endswith(".bnd") or file.endswith(".landmark"):
                    file_path = emotion_path+file
                    points = np.loadtxt(emotion_path+file, usecols=(1, 2, 3), encoding='utf-8') #x,y,z
                    X.append(points)
                    Y.append(label)
    X = np.array(X)
    Y = np.array(Y)
    X = X.reshape(X.shape[0], -1)    
    return X,Y  

def translate_data(X):
    X_translated = []
    for face in X:
        mean = np.mean(face)
        translated_face = face - mean
        X_translated.append(translated_face)
    X_translated = np.array(X_translated)
    return X_translated

def rotate_data(X, axis='x'):
    pi=round(2*acos(0.0), 3)
    sin_angle = math.sin(pi) #approx 0
    cos_angle = math.cos(pi) #approx -1
    rotated_X=[]
    rotated_face=[]
    if axis == 'x':
        rotation_matrix = np.array([[1, 0, 0], [0, cos_angle, sin_angle], [0, -sin_angle, cos_angle ]])
    elif axis == 'y':
        rotation_matrix = np.array([[cos_angle, 0, -sin_angle], [0, 1, 0], [sin_angle, 0, cos_angle]])
    elif axis == 'z':
        rotation_matrix = np.array([[cos_angle, sin_angle, 0], [-sin_angle, cos_angle, 0], [0, 0, 1]])
    else:
        raise ValueError("Axis must be 'x', 'y', or 'z'")
    for face in X:
        new_face = face.reshape(-1,3)
        for point in new_face:
            rotated_face.append(np.dot(rotation_matrix, point.T))
        print(rotated_face.shape)
        rotated_X.append(rotated_face)
    rotated_X = np.array(rotated_X) 
    return rotated_X

In [3]:
def classification(X,Y, classifier_type='RF'):
    if classifier_type == 'RF':
        print("Classifier - Random Forest")
        clf = RandomForestClassifier()
    elif classifier_type == 'SVM':
        print("Classifier - Support Vector Machine")
        clf = SVC()
    elif classifier_type == 'TREE':
        clf = DecisionTreeClassifier()

    Y_pred = []
    test_indices = []
    cv = StratifiedKFold(n_splits=10)
    for (train,test) in cv.split(X,Y):
        clf.fit(X[train],Y[train])
        Y_pred.append(clf.predict(X[test]))
        test_indices.append(test)
    
    return Y,Y_pred,test_indices

def model_evaluation(pred, indices, y):
    finalPredictions = []
    groundTruth = []
    for p in pred:
        finalPredictions.extend(p)
    for i in indices:
        groundTruth.extend(y[i])
    print(confusion_matrix(finalPredictions, groundTruth))
    print("Precision: ", precision_score(groundTruth, finalPredictions, average='macro'))
    print("Recall: ", recall_score(groundTruth, finalPredictions, average='macro'))
    print("Accuracy: " , accuracy_score(groundTruth, finalPredictions))


In [4]:
path = './BU4DFE_BND_V1.1/'
#path = args[3]
X,Y = load_data(path)

classifier = 'RF'
data_type = 'x'
#X = rotate_data(X,'x')
#X = translate_data(X)
y_original, y_pred, indices = classification(X,Y, classifier_type=classifier)
print("Following are the evaluation metrics for", classifier,"classifier and", data_type, "data type")
#Performing Evaluation
model_evaluation(y_pred, indices, y_original)

Classifier - Random Forest


MemoryError: Unable to allocate 425. KiB for an array with shape (54460,) and data type int64